# A topic modeling analysis of V. Putin’s addresses transcripts
v2.0

It's been 2 years. Let's see what has changed.

In [2]:
import re
from urllib.request import Request, urlopen
import time
import csv
import html2text
import pymorphy2
import pandas as pd
import numpy as np
import scipy as sp
import sys
import string
from nltk.corpus import stopwords
import nltk
from gensim.models import ldamodel, TfidfModel, LdaMulticore, CoherenceModel
import gensim.corpora
import pickle

## Downloading the data

Dmitry Medvedev was President from the 7th of May 2008 till the 7th of May 2012. We omit the transcripts from that period. Moreover, these are already downloaded. So we should run the code to fetch the new ones only. 

The last time I downloaded 37 pages with transcripts from 2012-06-12 till 2020-04-19. We'll proceed from page 1 to 9, remove duplicates, pre-process the thing and append it to the previous file. Then, we'll go to another .ipynb file... otherwise, it'll be to hard to read.

In [3]:
data = pd.read_csv('texts.csv')

In [42]:
data.link.max()

'http://kremlin.ru/events/president/transcripts/copy/63222'

In [43]:
# parsing pages to get links with transcripts
links = []
pattern = r'(\"/events/president/transcripts/speeches/\d{5}\")'
# pattern for elements looking as following:
# <a href="/events/president/transcripts/speeches/69123" rel="bookmark" aria-hidden="true" aria-label="Текст статьи" class="tabs_article item medium" title="Текст статьи"><i></i> </a>
# no, they don't go like 69123 69122 69121...

for i in range(1,10):
    url = "http://kremlin.ru/events/president/transcripts/speeches/page/" + str(i)
    req = Request(url, headers={'User-Agent': 'Mozilla/5.0'})
    page = urlopen(req).read().decode("utf8")
    links.append(re.findall(pattern, page))
    print("Success with page №" + str(i))
    time.sleep(3)

Success with page №1
Success with page №2
Success with page №3
Success with page №4
Success with page №5
Success with page №6
Success with page №7
Success with page №8
Success with page №9


In [59]:
links_sorted = sorted(list(set([y for x in links for y in x])))
len(links_sorted)

180

In [50]:
links_sorted[0]

'"/events/president/transcripts/speeches/62732"'

In [71]:
for link in links_sorted:
    if "http://kremlin.ru/events/president/transcripts/copy/" + link[-6:-1] in list(data.link):
        print(link)
        links_sorted.remove(link)

In [72]:
len(links_sorted)

162

In [73]:
texts = {}
i=0
for link in links_sorted:
    url = "http://kremlin.ru/events/president/transcripts/copy/" + link[-6:-1]
    h = html2text.HTML2Text()
    try:
        req = Request(url, headers={'User-Agent': 'Mozilla/5.0'})
        page = urlopen(req).read().decode("utf8")
        texts[url] = h.handle(page)
        i+=1
        print("Success with link №" + str(i))
        if i%15==0:
            time.sleep(45)
    except:
        print("Error")
        continue

Success with link №1
Success with link №2
Success with link №3
Success with link №4
Success with link №5
Success with link №6
Success with link №7
Success with link №8
Success with link №9
Success with link №10
Success with link №11
Success with link №12
Success with link №13
Success with link №14
Success with link №15
Success with link №16
Success with link №17
Success with link №18
Success with link №19
Success with link №20
Success with link №21
Success with link №22
Success with link №23
Success with link №24
Success with link №25
Success with link №26
Success with link №27
Success with link №28
Success with link №29
Success with link №30
Success with link №31
Success with link №32
Success with link №33
Success with link №34
Success with link №35
Success with link №36
Success with link №37
Success with link №38
Success with link №39
Success with link №40
Success with link №41
Success with link №42
Success with link №43
Success with link №44
Success with link №45
Success with link №

We have 162 new texts

In [74]:
print("We have ", len(texts) , " new texts!")

We have  162  new texts!


Saving as csv for safety reasons since download takes a lot of time:

In [75]:
f = open("texts_new.csv", "w")
writer = csv.writer(f)
writer.writerow(["link","text"])
for key, value in texts.items():
    writer.writerow([key, value])
f.close()

## Preparing the data 

In [100]:
data = pd.read_csv('texts_new.csv')
data_text = data[['text']]
#empty columns for future 
data_text["date"] = np.nan
data_text["tokens"] = np.nan
data_text["lemmas"] = np.nan
data_text = data_text.astype('str')

In [101]:
data_text.loc[1,'text']

'75-летие Победы 2020-05-09 11:50:00 Москва В день 75-летия Победы в Великой\nОтечественной войне Президент России возложил цветы к Могиле Неизвестного\nСолдата в Александровском саду. В завершение церемонии перед мемориалом\nторжественным маршем прошли рота почётного караула и оркестр. Глава\nгосударства также возложил цветы к обелискам городов-героев и памятному знаку\nв честь городов, удостоенных почётного звания «Город воинской славы». По\nокончании церемонии с Ивановской площади Кремля Владимир Путин наблюдал за\nвоздушным парадом в честь 75-летия Победы. Президент поручил Министру обороны\nСергею Шойгу передать благодарность лётному составу и наземным службам\nМинистерства обороны Российской Федерации, которые проводили и обеспечивали\nвоздушный парад в Москве и других городах России в День Победы. Владимир Путин\nпоздравил ветеранов, всех граждан России с Днём Победы. * * * Выступление на\nцеремонии возложения цветов к Могиле Неизвестного Солдата В.Путин: Уважаемые\nграждане Рос

Removing stop words, punctuation, numbers and dates (which go to a separate column):

In [102]:
stop_words = stopwords.words('russian')
stop_words.extend(['здравствуйте','уважаемая','уважаемый','уважаемые','товарищи','дамы', 'господа', 'коллеги', 'друзья','всё','что', 'это', 'так', 'вот', 'быть', 'как', 'в', '—', '–', 'к', 'на', '...','«'])


In [105]:
pattern = r'(\d{4}-\d{2}-\d{2})'

for idx in range(len(data_text)):
    text = data_text.loc[idx,'text']
    text = text.lower()

    date = re.search(pattern,text).group(0)
    data_text.at[idx,'date'] = date
    
    text = re.sub(r'\[.*?\]', '', text)
    text = re.sub(r'\n', ' ', text)
    text = re.sub(r'[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub(r'\w*\d\w*', '', text)
    data_text.at[idx, 'tokens'] = [word for word in text.split(' ') if word not in stop_words and word != ""]
    

Separating the speeches from titles and lemmatizing them if their POS tags interest us (we're leaving nouns, verbs, both forms of adjectives with comparatives, participles and gerunds).

In [107]:
morph = pymorphy2.MorphAnalyzer() #russian by default
for idx in range(len(data_text)):
    text = data_text.loc[idx,'tokens']
    for i in reversed(range(len(text))):
        if text[i]=="впутин":
            text = text[i+1:]
            break
    data_text.at[idx,'tokens'] = text
    
    lemmas = []
    for token in text:
        p = morph.parse(token)[0]
        lemma = p.normal_form
        if p.tag.POS in ['VERB','NOUN','COMP','INFN','ADJF','ADJS','PRTF','PRTS','GRND']:
            if len(lemma)>3:
                lemmas.append(lemma)
    data_text.at[idx,'lemmas'] = lemmas

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pymorphy2/units/base.py:70: DeprecationWarning: inspect.getargspec() is deprecated since Python 3.0, use inspect.signature() or inspect.getfullargspec()
  args, varargs, kw, default = inspect.getargspec(cls.__init__)
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pymorphy2/units/base.py:70: DeprecationWarning: inspect.getargspec() is deprecated since Python 3.0, use inspect.signature() or inspect.getfullargspec()
  args, varargs, kw, default = inspect.getargspec(cls.__init__)
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pymorphy2/units/base.py:70: DeprecationWarning: inspect.getargspec() is deprecated since Python 3.0, use inspect.signature() or inspect.getfullargspec()
  args, varargs, kw, default = inspect.getargspec(cls.__init__)
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pymorphy2/units/base.py:70: Deprecati

In [108]:
data_text

,text,date,tokens,lemmas
0,Обращение к австрийским гражданам по случаю 75...,2020-05-08,"[дорогие, liebe, freunde, добрый, день, grüß, ...","[дорогой, добрый, день, хотеть, поблагодарить,..."
1,75-летие Победы 2020-05-09 11:50:00 Москва В д...,2020-05-09,"[граждане, россии, ветераны, дорогие, сердца, ...","[гражданин, россия, ветеран, дорогой, сердце, ..."
2,Смотр Президентского полка 2020-05-09 12:15:00...,2020-05-09,"[поздравляю, военнослужащих, россии, наших, до...","[поздравлять, военнослужащий, россия, дорогой,..."
3,Поздравление выпускникам высших военно-учебных...,2020-05-10,"[офицеры, дорогие, поздравляю, выпускников, во...","[офицер, дорогой, поздравлять, выпускник, воен..."
4,Акция «Бессмертный полк» 2020-05-09 15:00:00 М...,2020-05-09,"[дорогие, хочу, обратиться, регулярно, зову, с...","[дорогой, хотеть, обратиться, звать, сердце, п..."
...,...,...,...,...
157,"Торжественный концерт, посвящённый 1160-летию ...",2022-09-21,"[сегодня, отмечаем, лет, российской, государст...","[отмечать, российский, государственность, позд..."
158,"Подписание договоров о принятии ДНР, ЛНР, Запо...",2022-09-30,"[граждане, россии, граждане, донецкой, луганск...","[гражданин, россия, гражданин, донецкий, луган..."
159,Митинг-концерт «Выбор людей. Вместе навсегда» ...,2022-09-30,"[дорогие, сегодня, особый, торжественный, всяк...","[дорогой, особый, торжественный, всякий, данны..."
160,Поздравление с Днём работника сельского хозяйс...,2022-10-09,"[дорогие, поздравляю, профессиональным, праздн...","[дорогой, поздравлять, профессиональный, празд..."


In [109]:
pickle.dump(data_text, open('data_text_new.dat', 'wb'))

Now, let's hope we will never open this .ipynb again